In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
from itertools import product
from itertools import permutations
from itertools import combinations
from pyEDM import *
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin
import time
import os
from copy import deepcopy
import math
import random
from sklearn.metrics import root_mean_squared_error
from scipy.stats import ttest_ind
import pickle
import ast
import json



from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))

import warnings
warnings.filterwarnings("ignore", 
    message="A worker stopped while some jobs were given to the executor.",
    module="joblib.externals.loky.process_executor")

In [2]:
def get_block(data, num_lags=1, tau=1):
    ''' Get a dataframe with all the possible valid lags of the variables. '''
    
    block = pd.concat([data[var].shift(lag*tau).rename(f'{var}(t-{lag*tau})') for lag in range(num_lags+1) for var in data.columns], axis=1)

    return block
def get_xmap_results_smap(block, target, embeddings, Tp, theta, lib, pred):
    '''Function to do exhaustive search of embeddings.'''
    
    def compute_rho(block, target, embedding, Tp, theta, lib, pred):
        xmap = SMap(dataFrame=block, target=target, columns=embedding, Tp=Tp, theta=theta, embedded=True, lib=lib, pred=pred, noTime=True)
        rho = xmap['predictions'][['Observations', 'Predictions']].corr().iloc[0,1]
        return embedding, xmap['predictions'], rho

    xmap_results = pd.DataFrame(columns=['embedding', 'rho'])
    xmap_results = Parallel(n_jobs=-1)(delayed(compute_rho)(block, target, embedding, Tp, theta, lib, pred) for embedding in embeddings)
    xmap_results = pd.DataFrame(xmap_results, columns=['embedding', 'result', 'rho'])
    xmap_results = xmap_results.sort_values(by='rho', ascending=False).reset_index(drop=True)
    
    return xmap_results

def get_valid_lags_tau(block, target, tau, num_lags, system_variables):
    
    # Get lags of system variables
    system_variable_lags = []
    for var in system_variables:
        var_lags = [f'{var}(t{i})' if i < 0 else f'{var}(t-{i})' for i in range(num_lags * tau, 0)]
        var_lags = var_lags[::tau][:num_lags]
        system_variable_lags = system_variable_lags + var_lags
    
    # Remove (t-0) lag of target variable from valid_lags
    valid_lags = [x for x in system_variable_lags if x[-4:-1] != f'{target}(t-1)']

    print(f'Valid lags are {valid_lags}')        
    return valid_lags


def create_single_model(E,theta,target,i_cols,lib, pred,HAB_embed,showPlot=False):
    driver = f'{target}(t-0)'
    #print(i_cols)
    cols = i_cols + [f'{target}(t-1)']
    #print(cols)
    result = SMap(
        dataFrame = HAB_embed, 
        columns = cols,
        target = driver,
        lib = lib,  # Library from rows 0 to 700
        pred = pred,
        E = E+1,
        theta=theta,
        noTime=True,
        showPlot = showPlot,
        embedded=True,
        ignoreNan = True
    )
    return result

def thresh_bloom_binary_prediction(obs,pred,threshold=8.03199999999999):
    #obs_bloom_95 = np.percentile(obs, 95) #incorrect
    #pred_bloom_95 = np.percentile(pred, 95) #incorrect
    obs_blooms = obs > threshold
    pred_blooms = pred > threshold
    Accuracy = 1 - (obs_blooms ^ pred_blooms).mean()
    True_pos = (obs_blooms & pred_blooms).sum() / obs_blooms.sum()
    False_pos = ((~obs_blooms) & pred_blooms).sum() / (~obs_blooms).sum()
    True_neg = ((~obs_blooms) & (~pred_blooms)).sum() / (~obs_blooms).sum()
    False_neg = (obs_blooms & (~pred_blooms)).sum() / obs_blooms.sum()
    
    return [Accuracy, True_pos, False_pos, True_neg, False_neg]


def create_model(data,system_variables,target,Tp,num_lags,tau,theta_list,lib,pred):
    HAB_embed_block = get_block(data,50)
    valid_lags = get_valid_lags_tau(HAB_embed_block, target, tau, num_lags, system_variables)
    HAB_embeddings = {}
    for E in range(3,8):
        # Get random embeddings using valid lags
        embeddings = set()
        sample = 800
        max_trials = 800
        trials = 0
        while len(embeddings) < sample and trials < max_trials:
            embedding = tuple(random.sample(valid_lags, E))
            sorted_embedding = tuple(sorted(embedding))
            if sorted_embedding not in embeddings:
                embeddings.add(sorted_embedding)
            trials += 1
        embeddings = [list(embedding) for embedding in embeddings]
        HAB_embeddings['{0}'.format((target, E))] = embeddings
        E_list = range(3,8)


        
    total_iterations = len(E_list) * len(theta_list)

    parameters = pd.DataFrame(columns=['target', 'columns', 'E', 'theta', 'rho', 'rmse', 'pred'])

    with tqdm(total=total_iterations) as pbar:
        for E, theta in product(E_list, theta_list):

            key = [key for key in HAB_embeddings.keys() if eval(key)[0] == target and eval(key)[1] == E] #HAB_embeddings["('Avg_Chloro', 4, 0, 6)"][0]
            embeddings = HAB_embeddings[key[0]]
            for embedding in embeddings:
                smap_model = create_single_model(E,theta,target,embedding,lib, pred,HAB_embed_block,showPlot=False)
                df = smap_model['predictions']
                rho = df[['Observations', 'Predictions']].corr().iloc[0,1]
                rmse = root_mean_squared_error(df['Observations'].iloc[1:-1], df['Predictions'].iloc[1:-1])
                #bbp = thresh_bloom_binary_prediction(df['Observations'].iloc[1:-1],df['Predictions'].iloc[1:-1])

                new_row = {'target': target, 'columns': embedding, 'E': E,'theta':theta, 'rho':rho, 'rmse':rmse, 'pred':df['Predictions']}
                parameters.loc[len(parameters)] = new_row

            

            pbar.update(1)
    parameters.sort_values(by='rho',ascending=False)
    return parameters

In [3]:
def ensemble_binary_bloom(parameters_df,n=300,p=0.05,samp=1,bloom_thresh=8.013):
    parameters_df = parameters_df.iloc[:n*samp].sample(n)
    sum = np.zeros(np.array(parameters_df['pred'].iloc[0][1:]).size)
    for i in range(n):
        curr = np.array(parameters_df['pred'].iloc[i][1:]) > bloom_thresh#np.percentile(parameters_df['pred'].iloc[i].iloc[1:],95)#
        sum = sum + curr
    return sum > (n*p)

'''
@parameters
data - dataframe of data containing column for target and desired system variables'
system variables - variables contained in system to be used for prediction
target - variable that will be forecasted (also used in prediction)
Tp - IDK
E - number of lags we go back
tau - step size that we go back at
theta_list - list of theta values to be used in ensemble model
lib - library used for prediction
pred - prediction length

@return
returns forecast for next time step given the dataframe 
'''

def next_forecast(data,system_variables,target,Tp,E,tau,theta_list,lib,pred,p=0.05):
    parameters = create_model(data,system_variables,target,Tp,E,tau,theta_list,lib,pred)
    return parameters
    


# Testings

In [4]:

paper_data = pd.read_csv('Data/orig_block.csv') #DESIRED DATAPATH
cols_to_drop = [col for col in paper_data.columns if 
                '1wk' in col or
                '_2wk' in col]
paper_data = paper_data.drop(columns = cols_to_drop)


# paper_data = paper_data.set_index('time')
paper_data['Time'] = paper_data.index.astype(int)
# paper_data['Avg_Chloro'] #= paper_data['Avg_Chloro'].apply(np.log1p) #LOG AMPUTATION
#IMPUTE HAB DATA
#Build basic linear regression model as sanity check
# Custom impute missing values with the average of the value in front and behind of it 
class ForwardBackwardImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
        X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')

        return (X_filled_forward + X_filled_backward) / 2


Imputer = ForwardBackwardImputer()
paper_data = paper_data.apply(pd.to_numeric, errors='coerce')
Imputer.fit(paper_data)
# paper_data = Imputer.transform(paper_data)#COMMENT OUT IF DONT WANT MEAN MPUTE
paper_data
HAB_embed = get_block(paper_data,50)
HAB_embed

,serial_day(t-0),chlA(t-0),nitrate(t-0),phosphate(t-0),silicate(t-0),nitrite(t-0),nitrate_phosphate_ratio(t-0),N_star(t-0),Si_star(t-0),SST(t-0),...,phosphate(t-50),silicate(t-50),nitrite(t-50),nitrate_phosphate_ratio(t-50),N_star(t-50),Si_star(t-50),SST(t-50),WIND_SPD(t-50),U_WIND(t-50),Time(t-50)
0,725662,0.430,2.520,0.295,2.355,0.020,8.542373,-2.200,2.06,19.118428,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,725669,0.920,2.110,0.305,2.605,0.115,6.918033,-2.770,2.30,19.229762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,725676,0.615,1.915,0.370,0.450,0.020,5.175676,-4.005,0.08,19.083670,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,725683,0.655,0.155,0.330,1.500,0.020,0.469697,-5.125,1.17,18.651255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,725690,0.410,0.200,0.320,2.000,0.030,0.625000,-4.920,1.68,18.196674,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1836,734926,1.790,0.940,0.280,4.930,0.120,3.357143,-3.540,4.65,14.007738,...,0.41,0.04,0.04,1.951220,-5.76,-0.37,14.589881,3.262202,0.422999,1786.0
1837,734933,1.995,4.360,0.450,7.690,0.270,9.688889,-2.840,7.24,13.920575,...,0.28,0.01,0.01,0.678571,-4.29,-0.27,14.330357,4.166964,-0.254430,1787.0
1838,734940,8.210,1.490,0.310,4.710,0.110,4.806452,-3.470,4.40,13.516654,...,0.30,0.02,0.02,1.133333,-4.46,-0.28,13.673810,4.640476,-0.054776,1788.0
1839,734947,2.775,3.760,0.600,6.280,0.230,6.266667,-5.840,5.68,13.329387,...,0.13,0.01,0.01,2.000000,-1.82,-0.12,13.511918,4.807660,1.236447,1789.0


In [5]:
data = paper_data

system_variables = ['N_star',
 'SST',
 'U_WIND',
 'WIND_SPD',
 'chlA',
 'nitrate',
 'nitrite',
 'phosphate',
 'silicate'] #SYSTEM VARIABLES



target = 'chlA' #TARGET
system_variables = system_variables
Tp = 1
E = 6 # up to -5 
tau = -1
theta_list = [1,5,9,15,25,35,45]
#CHANGED so lib is 2/3 of data and pred is the rest 1/3
lib = '1 1227'
pred = '1228 1841'


preds = next_forecast(data,system_variables,target,Tp,E,tau,theta_list,lib,pred)
preds

Valid lags are ['N_star(t-1)', 'N_star(t-2)', 'N_star(t-3)', 'N_star(t-4)', 'N_star(t-5)', 'N_star(t-6)', 'SST(t-1)', 'SST(t-2)', 'SST(t-3)', 'SST(t-4)', 'SST(t-5)', 'SST(t-6)', 'U_WIND(t-1)', 'U_WIND(t-2)', 'U_WIND(t-3)', 'U_WIND(t-4)', 'U_WIND(t-5)', 'U_WIND(t-6)', 'WIND_SPD(t-1)', 'WIND_SPD(t-2)', 'WIND_SPD(t-3)', 'WIND_SPD(t-4)', 'WIND_SPD(t-5)', 'WIND_SPD(t-6)', 'chlA(t-1)', 'chlA(t-2)', 'chlA(t-3)', 'chlA(t-4)', 'chlA(t-5)', 'chlA(t-6)', 'nitrate(t-1)', 'nitrate(t-2)', 'nitrate(t-3)', 'nitrate(t-4)', 'nitrate(t-5)', 'nitrate(t-6)', 'nitrite(t-1)', 'nitrite(t-2)', 'nitrite(t-3)', 'nitrite(t-4)', 'nitrite(t-5)', 'nitrite(t-6)', 'phosphate(t-1)', 'phosphate(t-2)', 'phosphate(t-3)', 'phosphate(t-4)', 'phosphate(t-5)', 'phosphate(t-6)', 'silicate(t-1)', 'silicate(t-2)', 'silicate(t-3)', 'silicate(t-4)', 'silicate(t-5)', 'silicate(t-6)']


100%|██████████| 35/35 [1:40:51<00:00, 172.91s/it]


,target,columns,E,theta,rho,rmse,pred
0,chlA,"[chlA(t-6), nitrite(t-5), phosphate(t-3)]",3,1,0.266969,4.053932,0 NaN 1 2.866443 2 4.57000...
1,chlA,"[SST(t-4), U_WIND(t-5), WIND_SPD(t-2)]",3,1,0.258937,4.086273,0 NaN 1 3.950908 2 3.35527...
2,chlA,"[SST(t-2), nitrate(t-5), nitrite(t-2)]",3,1,0.271649,4.071881,0 NaN 1 4.327255 2 3.67795...
3,chlA,"[nitrate(t-4), phosphate(t-5), phosphate(t-6)]",3,1,0.245255,4.087891,0 NaN 1 2.912740 2 2.52097...
4,chlA,"[chlA(t-2), nitrate(t-4), phosphate(t-4)]",3,1,0.286682,4.080833,0 NaN 1 2.671938 2 3.09202...
...,...,...,...,...,...,...,...
27911,chlA,"[N_star(t-3), U_WIND(t-4), WIND_SPD(t-1), chlA...",7,45,0.027706,44.088806,0 NaN 1 2.249084 2 -246...
27912,chlA,"[WIND_SPD(t-1), WIND_SPD(t-4), WIND_SPD(t-5), ...",7,45,0.021804,30.160609,0 NaN 1 4.281253 2 3.06732...
27913,chlA,"[N_star(t-1), SST(t-3), U_WIND(t-4), WIND_SPD(...",7,45,0.018652,25.422602,0 NaN 1 2.356850 2 4.31719...
27914,chlA,"[N_star(t-4), U_WIND(t-5), WIND_SPD(t-6), chlA...",7,45,0.019367,19.413878,0 NaN 1 7.580154 2 4.68755...


In [6]:
len(preds['pred'].iloc[0])

615

In [7]:
preds['pred'] = preds['pred'].apply(list)
preds.to_csv('deyle_models2.csv') #TO SAVE (BE CAREFUL ABOUT OVERWRITING)